In [1]:
!pip install -q transformers[torch] datasets evaluate sacrebleu sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

raw_ds = load_dataset("cfilt/iitb-english-hindi", split='train[:2000]')

raw_ds = raw_ds.train_test_split(test_size=0.2)

print (raw_ds["train"][10])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

{'translation': {'en': 'Current value', 'hi': 'मौजूदा मान'}}


# **Tokenization**

In [3]:
from transformers import AutoTokenizer

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
  input = [ex["en"] for ex in examples["translation"]]
  target = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(input, max_length=128, truncation=True)

  labels = tokenizer(target, max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


tokenized_ds = raw_ds.map(preprocess_function, batched=True)

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainer
raw_ds = load_dataset("cfilt/iitb-english-hindi", split='train[:2000]')

raw_ds = raw_ds.train_test_split(test_size=0.2)

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
  input = [ex["en"] for ex in examples["translation"]]
  target = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(input, max_length=128, truncation=True)

  labels = tokenizer(target, max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


tokenized_ds = raw_ds.map(preprocess_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
output_dir="./translation_model",
learning_rate=2e-5,
per_device_train_batch_size=16,
weight_decay=0.01,
save_total_limit=1,
num_train_epochs=3,
predict_with_generate=True,
fp16=True,
report_to="none"
)
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_ds["train"],
eval_dataset=tokenized_ds["test"],
data_collator=data_collator,
)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [15]:
trainer.train()

Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=300, training_loss=0.8386322530110677, metrics={'train_runtime': 40.5531, 'train_samples_per_second': 118.363, 'train_steps_per_second': 7.398, 'total_flos': 19110203228160.0, 'train_loss': 0.8386322530110677, 'epoch': 3.0})

In [26]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

if 'model' not in globals():
    model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

def translate(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt").to(device)
    output_tokens = model.generate(**inputs)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

test_sentence = "Today we are studing about Data Science and I am very interested in this subject"
print(f"English: {test_sentence}")
print(f"Hindi: {translate(test_sentence)}")

English: Today we are studing about Data Science and I am very interested in this subject
Hindi: आज हम डेटा विज्ञान के बारे में सत्रक कर रहे हैं... ... और मैं इस रण न्य में बहुत रुचि ले रहा हूँ


In [28]:
import sacrebleu

predictions = []
references = []

# Use the test split from your dataset
for item in raw_ds["test"]:

    src_sentence = item["translation"]["en"]
    tgt_sentence = item["translation"]["hi"]

    pred_sentence = translate(src_sentence)

    predictions.append(pred_sentence)
    references.append(tgt_sentence)

# Calculate BLEU
bleu = sacrebleu.corpus_bleu(predictions, [references])

print("Final BLEU Score:", bleu.score)

Final BLEU Score: 32.57816653849069
